In [ ]:
# Bibliotheken abrufen
import json
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime, timedelta
%matplotlib inline
sns.set()

In [ ]:
# URL als Variable definieren
url = "https://rodare.hzdr.de/api/records/?page=1&sort=mostrecent&size=1000"

In [ ]:
# Funktion um Inhalt aus URL in Variable r zu speichern
r = requests.get(url)

In [ ]:
# JSON lesen lassen
json = r.json()

In [ ]:
# Dictionary anlegen und Keys der gewünschten Werte definieren
Meta_dict = {}
Meta_dict['type'] = []
Meta_dict['license'] = []
Meta_dict['rel_id'] = []
Meta_dict['pub_date'] = []
Meta_dict['access_right'] = []
Meta_dict['downloads'] = []
Meta_dict['u_downloads'] = []
Meta_dict['u_views'] = []
Meta_dict['views'] = []
Meta_dict['ref_by'] = []
Meta_dict['doi'] = []

In [ ]:
# Definition einer Teilliste und for-Schleife, um den type der Records aus allen Einträgen auszulesen und aufzulisten
for record in json['hits']['hits']:
    Meta_dict['type'].append(record['metadata']['resource_type']['type'])
# Definition einer if-else Bedingung für den Abruf der Lizenz (gibt es nicht bei closed Access Veröffentlichungen)    
    if 'license' in record['metadata'].keys():
        Meta_dict['license'].append(record['metadata']['license']['id'])
    else:
        Meta_dict['license'].append('NaN')
# Iteration in Liste der related identifier anlegen und dort die Bedigungen für das Auslesen definieren
    count_1 = 0
    count_2 = 0
    for identifier in record['metadata']['related_identifiers']:
# nur wenn die Relation 'isReferencedBy' mit dem dazugehörigen Inhalt hzdr gefunden wird, wird der count_1 betätigt
# Hintergrund: Zählung aus RODARE-Sicht ob Record den Archivierungsworkflow von ROBIS bedient
        if identifier['relation'] == 'isReferencedBy' and 'hzdr' in identifier['identifier']:
            count_1 = count_1 + 1
# nur wenn die Relation 'isReferencedBy' gefunden wird, wird der count_2 betätigt
# Hintergrund: Zählung aus RODARE-Sicht ob Record überhaupt ein zugehörige Referenz hat (ROBIS + x)            
        if identifier['relation'] == 'isReferencedBy':
            count_2 = count_2 + 1
    Meta_dict['rel_id'].append(count_1)
    Meta_dict['ref_by'].append(count_2)
# einfaches Auslesen der Metadaten und Zuordnung in entsprechende Listen/Keys
    # Publikationsdatum nach 10 Zeichen abgeschnitten
    Meta_dict['pub_date'].append(record['created'][:10])
    Meta_dict['access_right'].append(record['metadata']['access_right'])
    Meta_dict['downloads'].append(int(record['stats']['downloads']))
    Meta_dict['u_downloads'].append(int(record['stats']['unique_downloads']))
    Meta_dict['u_views'].append(int(record['stats']['unique_views']))
    Meta_dict['views'].append(int(record['stats']['views']))
    Meta_dict['doi'].append(record['links']['doi'])

In [ ]:
# Umwandlung des Dictionaries in ein DataFrame mit pd
df = pd.DataFrame(Meta_dict)

In [ ]:
# Abbildung des Dataframes
df

In [ ]:
# Anzeige der Allgemeinen Eigenschaften der Spalten
df.describe(include='all')

In [ ]:
# Defintion und Berechungen zu Datumsangaben
aktuellesDatum = date.today()
rodare_pub = date(2018,2,28)
zeitraum = aktuellesDatum - rodare_pub
zr = zeitraum.days
ds = zr/len(df)
lpd = df['downloads'].sum()/zr

In [ ]:
# Ausgabe einige Sonderfälle und statistischer Werte
print('Stand',aktuellesDatum.strftime("%d.%m.%Y"),
      '\nSeit der Inbetriebnahme (28.02.2018) von RODARE sind',zeitraum.days,'Tage vergangen.\nInsgesamt wurden',
      str(len(df)),'Records veröffentlicht und',str(df['downloads'].sum()),
      'mal gedownloaded.\nDas entspricht: 1 Veröffentlichung alle',round(ds,1),
      'Tage und' ,round(lpd,1), 'Downloads pro Tag. \nDer meistgeladene Record',
       df.iloc[df.downloads.idxmax(), 10], 'wurde',df['downloads'].max(),
      'mal gedownloaded. \nDer neueste Download',df.iloc[0, 10],
      'wurde am',df['pub_date'].max(),'veröffentlicht. \nDer meistangesehene Record',
      df.iloc[df.views.idxmax(), 10],'wurde',df['views'].max(),'mal angesehen. \nIm Durchschnitt wird 1 Record',
      round(df['views'].mean(),1),'mal angesehen und',round(df['downloads'].mean(),1),
      'heruntergeladen. \nDie Standardabweichung für Downloads beträgt',round(df['downloads'].std(),1)
      ,'und die für Views beträgt',round(df['views'].std(),1),'. ')
# errechnete Korrelation zwische Views und Downloads
if df['downloads'].corr(df['views']) >= 0:
    print('Da die Korrelation zwischen Views und Downloads mit',round(df['downloads'].corr(df['views']),1),
          'größer als 0 ist, herrscht ein positiver Zusammenhang zwischen beiden Werten.')

In [ ]:
# absolute Aufschlüsselung der Datentypen nach Anzahl
dt=df['type'].value_counts()
print(dt)

In [ ]:
#Reduzierung der Datengrundlage und dort Werte ausgeben
df[df['type']=='dataset'].groupby(['license']).size()

In [ ]:
# Anzeige Menge Record-Software mit meisten Downloads
df.downloads[df['type']=='software'].max()

In [ ]:
# Anzeige Menge Record-Dataset mit meisten Downloads
df.downloads[df['type']=='dataset'].max()

In [ ]:
# relative Aufschlüsselung der Datentypen nach Anzahl
df['type'].value_counts(normalize=True)

In [ ]:
# Ausgabe der absoluten Verteilung von Referenzrecords, die in ROBIS publiziert wurden
df['rel_id'].value_counts()

In [ ]:
# Ausgabe der absoluten Verteilung von Referenzrecords
df['ref_by'].value_counts()

In [ ]:
# Verteilung der Referenzlinks pro Datentyp
df.groupby(['type','ref_by']).size()

In [ ]:
# absolute Aufschlüsselung der Lizenzen in Abhängigkeit des Datentypen mit Anzahl
df.groupby(['type','license']).size()

In [ ]:
# Verteilung Lizenz pro Datentyp mit Downloads
sns.set(rc={"figure.figsize":(12, 12)})
sns.relplot(x='type', y='license', size='downloads', data=df, height=3, aspect=2)
plt.savefig("./Pictures/01_Verteilung_Lizenz_Datentyp_Downloads.png", dpi=300)

In [ ]:
# Verteilung Lizenz pro Access-Recht mit Downloads
sns.set(rc={"figure.figsize":(12, 12)})
sns.relplot(x='access_right', y='license', size='downloads', data=df, height=5, aspect=1)
plt.savefig("./Pictures/02_Verteilung_Lizenz_AR_Downloads.png", dpi=300)

In [ ]:
# graphische Darstellung der Downloadverteilung
sns.set(rc={"figure.figsize":(12, 6)})
sns.distplot(df['downloads'], bins=20)
plt.savefig("./Pictures/03_Verteilung_Downloads.png", dpi=300)

In [ ]:
# graphische Darstellung der Viewverteilung in Relation zur Anzahl der Records
sns.set(rc={"figure.figsize":(12, 6)})
sns.distplot(df['views'],bins=60)
plt.savefig("./Pictures/04_Verteilung_Views.png", dpi=300)

In [ ]:
# graphische Darstellung der Views je Record in chronologischer Reihenfolge für bestimmten Zeitraum
# hier habe ich leider keine andere Auflösung oder Verlängerung der Achse hinbekommen(Xticks)
start_date = '2021-01-01'
plt.plot(df['pub_date'][df['pub_date'] >= start_date], 
         df['views'][df['pub_date'] >= start_date],
         color='#4870a0',
         marker='o')
plt.xlabel('Publikationsdatum')
plt.ylabel('Views')
plt.grid(True)
# Dreht Achsenbeschriftung
plt.gcf().autofmt_xdate()
plt.savefig("./Pictures/05_Verteilung_Views_2021.png", dpi=300)

In [ ]:
#Darstellung der Views und unique Views im Zeitverlauf
plt.stackplot(df['pub_date'], 
              df['u_views'], 
              df['views'],
              alpha=0.75,
              baseline='zero',
              colors=['#7289da','#f29fa9'],
              labels=['u_views', 'views'],
              )
plt.gcf().autofmt_xdate()
plt.savefig("./Pictures/06_Verteilung_Views_uniqueViews.png", dpi=300)

In [ ]:
# Darstellung Views pro Datentyp mit Anzeige der unique Views
sns.set(rc={"figure.figsize":(12, 12)})
sns.relplot(y='type', x='views',size='u_views', data=df)
plt.savefig("./Pictures/07_Verteilung_Views_uniqueViews_Datentyp.png", dpi=300)

In [ ]:
# Darstellung Downloads pro Datentyp mit Anzeige der unique Downloads
sns.relplot(y='type', x='downloads',size='u_downloads', data=df)
plt.savefig("./Pictures/08_Verteilung_Downloads_uniqueDownloads_Datentyp.png", dpi=300)

In [ ]:
# Darstellung Views und Downloads pro Datentyp 
sns.relplot(x='views', y='type', 
            size='downloads',
            data=df,
            height=3, aspect=3)
plt.savefig("./Pictures/09_Verteilung_Views_Downloads_Datentyp.png", dpi=300)